# CSE 316 code in python

In [2]:
import random

MEMORY_SIZE = 1000
BLOCK_SIZE = 10
NUM_BLOCKS = MEMORY_SIZE // BLOCK_SIZE
NUM_PROCESSES = 50
SIMULATION_TIME = 100

class MemoryBlock:
    def __init__(self, start_index):
        self.start_index = start_index
        self.end_index = start_index + BLOCK_SIZE
        self.process_id = None
    
    def is_free(self):
        return self.process_id is None

class MemoryManager:
    def __init__(self, allocation_algorithm):
        self.blocks = [MemoryBlock(i * BLOCK_SIZE) for i in range(NUM_BLOCKS)]
        self.free_list = [b for b in self.blocks if b.is_free()]
        self.allocation_algorithm = allocation_algorithm
    
    def allocate(self, process_id):
        block = self.allocation_algorithm(self.free_list)
        if block is None:
            return None
        block.process_id = process_id
        self.free_list.remove(block)
        return block
    
    def deallocate(self, block):
        block.process_id = None
        self.free_list.append(block)
        self.free_list.sort(key=lambda b: b.start_index)
    
def first_fit(free_list):
    for block in free_list:
        if block.end_index - block.start_index >= BLOCK_SIZE:
            return block
    return None

def best_fit(free_list):
    best_block = None
    for block in free_list:
        if block.end_index - block.start_index >= BLOCK_SIZE:
            if best_block is None or block.end_index - block.start_index < best_block.end_index - best_block.start_index:
                best_block = block
    return best_block

def worst_fit(free_list):
    worst_block = None
    for block in free_list:
        if block.end_index - block.start_index >= BLOCK_SIZE:
            if worst_block is None or block.end_index - block.start_index > worst_block.end_index - worst_block.start_index:
                worst_block = block
    return worst_block

def simulate_memory_management(allocation_algorithm):
    memory_manager = MemoryManager(allocation_algorithm)
    
    fragmentation = 0
    num_wasted_blocks = 0

    for time_unit in range(SIMULATION_TIME):
        for i in range(NUM_PROCESSES):
            block = memory_manager.allocate(i)
            if block is None:
                num_wasted_blocks += 1
        for block in memory_manager.blocks:
            if block.process_id is not None and random.random() < 0.1:
                memory_manager.deallocate(block)
    
        num_free_blocks = len(memory_manager.free_list)
        fragmentation += num_free_blocks / NUM_BLOCKS
    avg_fragmentation = fragmentation / SIMULATION_TIME
    avg_wasted_blocks = num_wasted_blocks / SIMULATION_TIME
    
    return (avg_fragmentation, avg_wasted_blocks)

print("First-Fit:")
print(simulate_memory_management(first_fit))
print("Best-Fit:")
print(simulate_memory_management(best_fit))
print("Worst-Fit:")
print(simulate_memory_management(worst_fit))

First-Fit:
(0.10420000000000001, 39.22)
Best-Fit:
(0.10859999999999999, 38.77)
Worst-Fit:
(0.10549999999999997, 39.12)
